<a href="https://colab.research.google.com/github/harika140901/ee5333/blob/main/EE5333_tutorials-master/drc/Design_Rule_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --break-system-packages --force https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp311-cp311-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.8/617.8 kB 7.5 MB/s eta 0:00:00


In [3]:
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/Nangate.lef
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/drc/rects.txt

--2025-03-19 06:45:28--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/Nangate.lef
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1083933 (1.0M) [text/plain]
Saving to: ‘Nangate.lef’

Nangate.lef         100%[===================>]   1.03M  --.-KB/s    in 0.06s   

2025-03-19 06:45:28 (15.9 MB/s) - ‘Nangate.lef’ saved [1083933/1083933]

--2025-03-19 06:45:28--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/drc/rects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208141 (203K) [

In [4]:
# find parallel run length between two rectangles
# smaller dimension is assumed to be the width
def getPRL(r1, r2):
  w1, h1 = r1.ur.x - r1.ll.x, r1.ur.y - r1.ll.y
  w2, h2 = r2.ur.x - r2.ll.x, r2.ur.y - r2.ll.y
  if w1 > h1 and w2 > h2:
    if r1.ur.x > r2.ll.x and r2.ur.x > r1.ll.x:
      return min(r1.ur.x - r2.ll.x, r2.ur.x - r1.ll.x)
    return 0
  if r1.ur.y > r2.ll.y and r2.ur.y > r1.ll.y:
    return min(r1.ur.y - r2.ll.y, r2.ur.y - r1.ll.y)
  return 0

In [5]:
# get the spacing value from the spacing table for two input rectangles
def getSpacing(layer, r1, r2):
  width1, width2, prl =   min(r1.ur.x - r1.ll.x, r1.ur.y - r1.ll.y), min(r2.ur.x - r2.ll.x, r2.ur.y - r2.ll.y), getPRL(r1, r2)
  spacing = layer.spacing()
  i = 0
  for i, w in enumerate(layer.spacingTable().width):
    if width1 < w:
      break
  ind1, i = i - 1, 0
  for i, w in enumerate(layer.spacingTable().width):
    if width2 < w:
      break
  ind2, i = i - 1, 0
  for i, p in enumerate(layer.spacingTable().prl):
    if prl < p:
      break
  indp = i - 1
  if ind1 >= 0 and ind2 >= 0 and indp >= 0:
    spacing = max(layer.spacingTable().spacing[ind1][indp], layer.spacingTable().spacing[ind2][indp], spacing)
  return spacing

In [6]:
# check if two rectangles violate spacing
def checkSpacing(r1, r2, s):
  r = LEFDEFParser.Rect(r1.ll.x - s, r1.ll.y - s, r1.ur.x + s, r1.ur.y + s) # bloat r1 by s on all sides
  return (r.ur.x > r2.ll.x and r.ll.x < r2.ur.x and r.ur.y > r2.ll.y and r.ll.y < r2.ur.y)

In [7]:
import LEFDEFParser
l = LEFDEFParser.LEFReader()
l.readLEF('Nangate.lef')
#for m in l.macros():
#    m.print()
layerMap = {layer.name():layer for layer in l.layers()}

'''
for k,v in layerMap.items():
  print(v.name(), v.width(), v.spacing())
  print('prl-> ', '\t\t', ' '.join([str(i) for i in v.spacingTable().prl]))
  for i, w in enumerate(v.spacingTable().width):
    #print('width ', v.spacingTable().width[i], '\t', ' '.join([str(i) for i in v.spacingTable().spacing[i]]))
'''

maxSpacing = {layer.name():max(layer.spacing(), max([max(s) for s in layer.spacingTable().spacing])  if len(layer.spacingTable().spacing) else 0) for layer in l.layers()}
print(maxSpacing)


{'metal1': 130, 'metal2': 3000, 'metal3': 3000, 'metal4': 3000, 'metal5': 3000, 'metal6': 3000, 'metal7': 3000, 'metal8': 3000, 'metal9': 3000, 'metal10': 3000}


In [8]:
layer = 'metal2'
rects = [LEFDEFParser.Rect(0, 0, 1000, 140), LEFDEFParser.Rect(400, 300, 1000, 440), LEFDEFParser.Rect(1100, 0, 1600, 140)]
for i in range(len(rects)):
  for j in range(i + 1, len(rects)):
    print(rects[i], rects[j], checkSpacing(rects[i], rects[j], getSpacing(layerMap[layer], rects[i], rects[j])))


[(0,0),(1000,140)] [(400,300),(1000,440)] False
[(0,0),(1000,140)] [(1100,0),(1600,140)] True
[(400,300),(1000,440)] [(1100,0),(1600,140)] False


In [9]:
with open('rects.txt', 'r') as fh:
  rects = []
  for l in fh:
    s = [int(i) for i in l.split()]
    if len(s) == 4:
      rects.append(LEFDEFParser.Rect(s[0], s[1], s[2], s[3]))
import time
for N in [100, 200, 500, 1000]:
  for layer in ['metal1', 'metal2']:
    count, t = 0, time.time()
# brute-force quadratic check between every pair of rectangles
    for i in range(min(N, len(rects))):
      for j in range(i + 1, min(N, len(rects))):
        if checkSpacing(rects[i], rects[j], getSpacing(layerMap[layer], rects[i], rects[j])): count += 1
    print('layer :', layer, '#violations :', count, 'runtime :', time.time() - t)
  print()


layer : metal1 #violations : 144 runtime : 0.3750331401824951
layer : metal2 #violations : 185 runtime : 0.46103644371032715

layer : metal1 #violations : 626 runtime : 1.1191556453704834
layer : metal2 #violations : 817 runtime : 1.4321706295013428

layer : metal1 #violations : 3931 runtime : 5.772539138793945
layer : metal2 #violations : 5160 runtime : 7.733041048049927

layer : metal1 #violations : 16165 runtime : 25.376755475997925
layer : metal2 #violations : 21088 runtime : 29.98246431350708



In [14]:
# build a kdtree of rectangles
# build the kd-tree using the input set of rectangles
class node:
  def __init__(self,name):
    self._name = name
    self._right = None
    self._left = None
    self._rects = []
    self._median = 0

class kdtree:
  def __init__(self, rects):
    Node = node(0)
    self.root = Node
    Node._left = node(1)
    Node._right = node(1)
    Node._rects = rects
    listN = [Node._left, Node._right]
    for i in [2,3,4]:
      newlist = []
      for n in listN:
        n._left = node(i)
        n._right = node(i)
        newlist.append(n._left)
        newlist.append(n._right)
      listN = newlist
    listN = [Node]
    p = 0
    for module in listN:
      p += 1
      rlist = module._rects
      if module._name == 0:
        median = sum(rlist[i].ll.x for i in range(len(rlist))) / len(rlist)
      elif module.name == 1:
        median = sum(rlist[i].ll.y for i in range(len(rlist))) / len(rlist)
      elif module._name == 2:
        median = sum(rlist[i].ur.x for i in range(len(rlist))) / len(rlist)
      elif module.name == 3:
        median = sum(rlist[i].ur.y for i in range(len(rlist))) / len(rlist)
      else:
        break
      module._median = median
      newList = []
      newList.append(module._left)
      newList.append(module._right)
      for i in range(len(rlist)):
        if module._name == 0:
          if rlist[i].ll.x < median:
            Node._left._rects.append(rlist[i])
          else:
            Node._right._rects.append(rlist[i])
        elif module._name == 1:
          if rlist[i].ll.y < median:
            Node._left._rects.append(rlist[i])
          else:
            Node._right._rects.append(rlist[i])
        elif module._name == 2:
          if rlist[i].ur.x < median:
            Node._left._rects.append(rlist[i])
          else:
            Node._right._rects.append(rlist[i])
        elif module._name == 3:
          if rlist[i].ur.y < median:
            Node._left._rects.append(rlist[i])
          else:
            Node._right._rects.append(rlist[i])
      if p == len(listN):
        listN = newList

  def region_query(self, region):
    l = []
    listN = [self.root]
    while len(listN) != 0:
      k = listN.pop()
      # if len(listN) == 0:
      #   bootToIncreD = True
      if k._name == 0 or k._name == 2:
        if region.ll.x < k._median and region.ur.x < k._median:
          listN.append(k._left)
        elif region.ll.x > k._median and region.ur.x > k._median:
          listN.append(k._right)
        else:
          listN.append(k._right)
          listN.append(k._left)
      elif k._name == 1 or k._name == 3:
        if region.ll.y < k._median and region.ur.y < k._median:
          listN.append(k._left)
        elif region.ll.y > k._median and region.ur.y > k._median:
          listN.append(k._right)
        else:
          listN.append(k._right)
          listN.append(k._left)
      else:
        for p in listN:
          if p._name == 4:
            l = l + p._rects
            listN.remove(p)
    return l

In [15]:
kdt = kdtree(rects)
for N in [100, 1000, 2000]:
  t = time.time()
  for layer in ['metal1', 'metal2', 'metal3']:
    count = 0
    for i in range(min(N, len(rects))):
      # replace pair-wise check with potentially conflicting neighbours through region query
      s = maxSpacing[layer]
      nbrs = kdt.region_query(LEFDEFParser.Rect(rects[i].ll.x - s, rects[i].ll.y - s, rects[i].ur.x + s, rects[i].ur.y + s))
      for j in range(len(nbrs)):
        if rects[i] == nbrs[j]: continue
        if checkSpacing(rects[i], nbrs[j], getSpacing(layerMap[layer], rects[i], nbrs[j])): count += 1
    print(layer, count)
  print('runtime : ', time.time() - t)

metal1 0
metal2 0
metal3 0
runtime :  0.010742902755737305
metal1 0
metal2 0
metal3 0
runtime :  0.09420466423034668
metal1 0
metal2 0
metal3 0
runtime :  0.20350337028503418


In [16]:
import time
for N in [100, 1000]:
  for layer in ['metal1', 'metal2', 'metal3']:
    count, t = 0, time.time()
# brute-force quadratic check between every pair of rectangles
    for i in range(min(N, len(rects))):
      for j in range(i + 1, min(N, len(rects))):
        if checkSpacing(rects[i], rects[j], getSpacing(layerMap[layer], rects[i], rects[j])): count += 1
    print('layer :', layer, '#violations :', count, 'runtime :', time.time() - t)
  print()

layer : metal1 #violations : 144 runtime : 0.2294478416442871
layer : metal2 #violations : 185 runtime : 0.28020763397216797
layer : metal3 #violations : 185 runtime : 0.3879544734954834

layer : metal1 #violations : 16165 runtime : 28.74310564994812
layer : metal2 #violations : 21088 runtime : 32.11531925201416
layer : metal3 #violations : 21088 runtime : 31.05124020576477

layer : metal1 #violations : 65562 runtime : 107.58234548568726
layer : metal2 #violations : 85559 runtime : 123.88293409347534
layer : metal3 #violations : 85559 runtime : 124.95951581001282

